In [ ]:
from google.colab import files
uploaded = files.upload()


In [4]:
import pandas as pd
df = pd.read_csv('ner_data.csv', encoding= 'unicode_escape')

In [5]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [7]:
from itertools import chain
def get_dict_map(df, token_or_tag):
  tok2idx = {}
  idx2tok = {}

  if token_or_tag == 'token':
    vocab = list(set(df['Word'].to_list()))
  else:
    vocab = list(set(df['Tag'].to_list()))

  idx2tok = {idx:tok for idx, tok in enumerate(vocab)}
  tok2idx = {tok:idx for idx, tok in enumerate(vocab)}
  return tok2idx, idx2tok

token2idx, idx2token = get_dict_map(df, 'token')
tag2idx, idx2tag = get_dict_map(df, 'tag')

In [8]:
df['Word_idx'] = df['Word'].map(token2idx)
df['Tag_idx'] = df['Tag'].map(tag2idx)
df_fillna = df.fillna(method = 'ffill', axis = 0)
df_group = df_fillna.groupby(['Sentence #'],as_index=False)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [11]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
def get_pad_train_test_val(df_group, df):
  
  #get maximum token and tag length
  n_token = len(list(set(df['Word'].to_list())))
  n_tag = len(list(set(df['Tag'].to_list())))

  #pad tokens
  tokens = df_group['Word_idx'].to_list()
  maxlen = max([len(s) for s in tokens])
  pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype ='int32', padding ='post', value = n_token - 1)

  #Pad Tags (y var) and convert it into one hot encoding
  tags = df_group['Tag_idx'].tolist()
  pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
  n_tags = len(tag2idx)
  pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

  #Split train, test and validation set
  tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2021)
  train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2021)

  print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
  return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(df_group, df)


train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [12]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [14]:
input_dim = len(list(set(df['Word'].to_list())))+1
output_dim = 68
input_length = max([len(s) for s in df_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [16]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [17]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
result = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 104, 68)           2392172   
_________________________________________________________________
bidirectional (Bidirectional (None, 104, 136)          74528     
_________________________________________________________________
lstm_1 (LSTM)                (None, 104, 68)           55760     
_________________________________________________________________
time_distributed (TimeDistri (None, 104, 17)           1173      
Total params: 2,523,633
Trainable params: 2,523,633
Non-trainable params: 0
_________________________________________________________________
26/26 [==============================] - 143s 5s/step - loss: 0.1264 - accuracy: 0.9679 - val_loss: 0.1358 - val_accuracy: 0.9678


In [20]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is Naveen Pandey \n I am from India \n I want to work with Google \n Steve Jobs is My Inspiration')
displacy.render(text, style = 'ent', jupyter=True)
